In [1]:
# General
import pandas as pd
import numpy as np
import datetime
import os
import glob
import sqlite3
from math import ceil

# Data viz
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

#pre-processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

#feature selection
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE,f_classif
from sklearn.linear_model import LogisticRegression


import warnings
warnings.filterwarnings('ignore')

# Importing data

In [2]:
df = pd.read_csv('../data/MACIMIDE_Global_Expatriate_Dual_Citizenship_Dataset_V5.00.csv')

try:
    wdi = pd.read_csv('../data/WDIData.csv',usecols = range(65))
except:
    #saves the WDIData table without extracting the zip file
    with zipfile.ZipFile('../data/WDI_csv.zip') as myzip:
        wdi = pd.read_csv(myzip.open('WDIData.csv'),usecols = range(65))
        country = pd.read_csv(myzip.open('WDICountry.csv'))
        ind = pd.read_csv(myzip.open('WDISeries.csv'))

# Cleaning data

1. Keeping only the countries in WDI data
2. Keep years from 2000 to 2020

In [3]:
# 1. Keep only countries
cname = wdi.drop_duplicates(subset='Country Name')['Country Name']
wdi = wdi.loc[~wdi['Country Name'].isin(cname[:49])]

# 2. Keep only years we want
df = df.query('Year >= 2000 & Year <2020')
wdi = pd.concat([wdi.iloc[:,:4],wdi.iloc[:,44:-1]],axis=1)

# Changing world Region and selecting some countries

In [4]:
country_old = df.loc[df.world_region.isna()]['country'].unique()
df = df.loc[~df['country'].isin(country_old)]
def world_region(x):
    if x == 1:
        return 'Africa'
    elif x == 2:
        return 'Asia'
    elif x == 3:
        return 'Europe'
    elif x== 6:
        return 'Oceania'
    else:
        return 'America'    
    
df['world_region'] = df['world_region'].apply(lambda x:world_region(x) )
df = df.query('Year >= 2000 & Year <2020')


In [5]:
America = ["United States of America",'Canada','Mexico','Brazil', 'Colombia','Chile','Argentina']
Europe = ['Italy', 'Germany', 'Denmark', 'Poland', 'United Kingdom (of Great Britain and Northern Ireland)','France','Netherlands']
Extreme_Orient = ['China',  'Thailand', 'Australia', 'India',  'Azerbaijan','Japan']
Africa = ['South Africa', 'Djibouti', 'Morocco', 'Nigeria', 'Botswana']
Countries = [c for i in (America,Europe,Africa,Extreme_Orient) for c in i]

df_country = df.copy()
df_country = df.loc[df['country'].isin((Countries))]
len(df_country['country'].unique())

25

In [6]:
wdi.head()

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
70707,Afghanistan,AFG,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,8.8,9.51,10.39,11.46,12.43,13.490000,...,20.6800,22.330000,24.080000,26.170000,27.990000,30.100000,32.44000,NaN,NaN,NaN
70708,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,22.295269,...,42.7000,43.222019,69.100000,68.982941,89.500000,71.500000,97.70000,97.700000,98.715622,97.70000
70709,Afghanistan,AFG,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,7.109005,...,30.2188,29.572881,60.849157,61.315788,86.500512,64.573354,97.09936,97.091973,98.309603,96.90219
70710,Afghanistan,AFG,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,74.000000,...,82.8000,86.567779,95.000000,92.773354,98.700000,92.500000,99.50000,99.500000,99.902145,100.00000
70711,Afghanistan,AFG,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,9.005013,NaN,NaN,9.961000,NaN,NaN,14.893312,NaN,NaN


# Missing Values Treatment

First we are going to filter indicators with more than 20% of the data as nan values along countries and years. Inputation in thoses cases would change too much the analysis

In [7]:
Indicator_names = np.mean(1 -wdi.groupby('Indicator Name').count().iloc[:,3:-1]/217,axis =1)
Indicator_little_nan = pd.DataFrame(Indicator_names[Indicator_names<.06].sort_values(),columns=['% of nan'])
Indicator_little_nan

,% of nan
Indicator Name,
Adjusted savings: mineral depletion (current US$),0.000000
"Population, total",0.001698
Population growth (annual %),0.001698
Land area (sq. km),0.002911
Population density (people per sq. km of land area),0.002911
Surface area (sq. km),0.005578
Urban population (% of total population),0.010914
Urban population growth (annual %),0.010914
Rural population,0.010914


In [8]:
# select indicators with less nan values
filtered_wdi = wdi.loc[wdi['Indicator Name'].isin(Indicator_little_nan.index)]

In [9]:
s = filtered_wdi.groupby(['Country Code','Indicator Name']).count().sum(axis = 1) / 22
pd.DataFrame(s,columns=["%nan"]).reset_index().groupby('Indicator Name').mean().sort_values("%nan")

,%nan
Indicator Name,
Arable land (hectares per person),0.909091
Arable land (hectares),0.909091
Arable land (% of land area),0.909091
Agricultural land (% of land area),0.919355
Agricultural land (sq. km),0.919355
Renewable energy consumption (% of total final energy consumption),0.938626
Capture fisheries production (metric tons),0.939045
Total fisheries production (metric tons),0.939045
Fixed telephone subscriptions (per 100 people),0.943653


Now, for the 396 indicator left we should try to inputate the missing values by some of the below methods:
- KNNImputer
- mode (by Population density and GDP per capta)
- mean (by Population density and GDP per capta)

the *'by Population density and GDP per capta'* suppose to find the best mode and mean along countries, otherwise inputation would be too bias. Both indicatiors have few nan values and take in consideration important factors in selecting similar countries.

In [10]:
np.sqrt(len(Indicator_little_nan))

6.708203932499369

In [11]:
# knn imputer
imputer = KNNImputer(n_neighbors=10)
filtered_wdi.iloc[:,4:] = imputer.fit_transform(filtered_wdi.iloc[:,4:])
filtered_wdi

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
70708,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,2.910849e+01,2.915522e+01,3.189710e+01,3.478334e+01,3.670975e+01,2.229527e+01,...,4.270000e+01,4.322202e+01,6.910000e+01,6.898294e+01,8.950000e+01,7.150000e+01,9.770000e+01,9.770000e+01,9.871562e+01,9.770000e+01
70710,Afghanistan,AFG,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,7.269276e+01,7.361219e+01,7.664915e+01,7.711550e+01,7.859403e+01,7.400000e+01,...,8.280000e+01,8.656778e+01,9.500000e+01,9.277335e+01,9.870000e+01,9.250000e+01,9.950000e+01,9.950000e+01,9.990215e+01,1.000000e+02
70738,Afghanistan,AFG,Adjusted savings: carbon dioxide damage (curre...,NY.ADJ.DCO2.CD,1.436535e+07,1.586448e+07,2.157660e+07,2.547320e+07,2.020204e+07,3.084251e+07,...,2.416302e+08,3.647899e+08,3.340286e+08,2.924376e+08,2.849291e+08,3.141359e+08,3.114902e+08,3.473530e+08,3.839152e+08,4.336361e+08
70744,Afghanistan,AFG,Adjusted savings: energy depletion (current US$),NY.ADJ.DNGY.CD,1.797047e+06,1.078196e+06,9.725494e+05,9.475434e+05,1.235225e+06,1.469289e+06,...,1.044313e+07,2.649364e+07,3.447720e+07,1.494763e+07,1.410024e+07,9.737235e+06,1.293113e+07,2.153331e+07,2.826238e+07,2.074859e+07
70747,Afghanistan,AFG,Adjusted savings: mineral depletion (current US$),NY.ADJ.DMIN.CD,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,3.654550e+05,4.721533e+05,3.051536e+05,3.888372e+05,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383679,Zimbabwe,ZWE,Surface area (sq. km),AG.SRF.TOTL.K2,3.907600e+05,3.907600e+05,3.907600e+05,3.907600e+05,3.907600e+05,3.907600e+05,...,3.907600e+05,3.907600e+05,3.907600e+05,3.907600e+05,3.907600e+05,3.907600e+05,3.907600e+05,3.907600e+05,3.907600e+05,3.981013e+05
383741,Zimbabwe,ZWE,Total fisheries production (metric tons),ER.FSH.PROD.MT,9.716300e+04,9.074200e+04,9.369500e+04,8.695700e+04,7.369000e+04,8.229600e+04,...,9.497700e+04,1.101170e+05,1.091610e+05,1.389890e+05,1.554850e+05,1.409990e+05,1.392870e+05,1.110100e+05,1.258820e+05,1.305048e+05
383808,Zimbabwe,ZWE,Urban population,SP.URB.TOTL,4.010951e+06,4.074399e+06,4.134392e+06,4.131349e+06,4.122108e+06,4.119361e+06,...,4.215138e+06,4.257061e+06,4.306228e+06,4.359432e+06,4.416224e+06,4.473872e+06,4.531238e+06,4.589452e+06,4.650597e+06,4.717307e+06
383809,Zimbabwe,ZWE,Urban population (% of total population),SP.URB.TOTL.IN.ZS,3.375800e+01,3.417000e+01,3.458500e+01,3.447900e+01,3.429400e+01,3.411000e+01,...,3.319600e+01,3.301500e+01,3.283400e+01,3.265400e+01,3.250400e+01,3.238500e+01,3.229600e+01,3.223700e+01,3.220900e+01,3.221000e+01


In [12]:
# mode

# mean

# Pre processing Data


In [13]:
#2019 data
wdi_pivoted = filtered_wdi.pivot(index='Country Code',columns='Indicator Name', values='2019')

dual_city_groups_2019 = df.loc[df.Year==2019].set_index('ISO3')[['Dualcit_grouped']]

wdi_scaled = wdi_pivoted.join(dual_city_groups_2019).dropna(subset=['Dualcit_grouped'])


scaler = MinMaxScaler()

wdi_scaled.iloc[:,:-1] = scaler.fit_transform(wdi_scaled.iloc[:,:-1])

# Selecting Features

### Filter Methods
- Kendalls correlation
- Anova correlation

### Wrapper methods
- RFE

### Assamble Methods
- random forest split

In [14]:
cor_kendall = wdi_scaled.corr(method ='kendall')
cor_kendall[['Dualcit_grouped']].sort_values('Dualcit_grouped',ascending=False)

,Dualcit_grouped
Dualcit_grouped,1.000000
"Access to electricity, urban (% of urban population)",0.105596
Agricultural land (% of land area),0.100209
Access to electricity (% of population),0.083226
Urban population (% of total population),0.063845
Fixed telephone subscriptions (per 100 people),0.054857
"Primary education, duration (years)",0.041614
People using at least basic sanitation services (% of population),0.037554
"Birth rate, crude (per 1,000 people)",0.031138
Lower secondary school starting age (years),0.024094


In [79]:
def kmeans_Hierarchical(considered_df,threshold=0.3):
    kmeans = KMeans(n_clusters=30, random_state=0).fit(considered_df)
    considered_df['Cluster'] = pd.Series(kmeans.labels_)
    kmeans_clusters = considered_df.groupby("Cluster").mean()
    considered_df.drop('Cluster',axis=1,inplace=True)
    
    # Hierarchical clustering assessment using scipy
    Z = linkage(kmeans_clusters, method='ward')
    sns.set()
    fig = plt.figure(figsize=(15,7))

    # Dendrogram
    set_link_color_palette(list(sns.color_palette().as_hex()))
    dendrogram(Z, color_threshold = threshold*max(Z[:,2]), orientation='top', no_labels=True, above_threshold_color='k')

    plt.hlines(threshold*max(Z[:,2]), 0, 1000, colors="r", linestyles="dashed")
    plt.title('Hierarchical Clustering - Ward\'s Dendrogram', fontsize=23)
    plt.xlabel('SOM Units', fontsize=13)
    plt.ylabel('Euclidean Distance', fontsize=13)
    plt.show()

In [82]:
wdi_scaled

,Access to electricity (% of population),"Access to electricity, urban (% of urban population)",Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: energy depletion (current US$),Adjusted savings: mineral depletion (current US$),Agricultural land (% of land area),Agricultural land (sq. km),Arable land (% of land area),Arable land (hectares per person),Arable land (hectares),...,Rural population,Rural population (% of total population),Rural population growth (annual %),"Secondary education, duration (years)",Surface area (sq. km),Total fisheries production (metric tons),Urban population,Urban population (% of total population),Urban population growth (annual %),Dualcit_grouped
Country Code,,,,,,,,,,,,,,,,,,,,,
AFG,0.975343,1.193129e-11,0.000059,2.849693e-06,0.000000,7.896164e-12,5.383713e-08,1.637648e-12,2.498870e-14,1.086765e-06,...,0.031544,0.855862,9.612330e-13,0.4,0.036126,0.000111,0.011616,0.144138,4.993755e-08,2.0
AGO,0.417560,8.136675e-12,0.000193,2.360897e-03,0.000000,6.252822e-12,7.889470e-08,5.163081e-13,1.488376e-14,6.915537e-07,...,0.012022,0.389890,8.701512e-13,0.4,0.069398,0.005980,0.024979,0.610110,5.963041e-08,2.0
ALB,1.000000,1.193129e-11,0.000027,1.712511e-05,0.000086,5.818182e-12,1.629188e-09,3.163008e-12,2.162635e-14,8.513614e-08,...,0.001236,0.446928,3.142181e-13,0.6,0.001590,0.000176,0.002067,0.553072,2.636044e-08,2.0
AND,1.000000,1.193129e-11,0.000003,1.000000e+00,0.000000,5.289095e-12,2.544867e-11,2.186556e-13,4.081885e-07,1.062490e-10,...,0.000010,0.138513,8.059332e-13,0.4,0.000025,0.000000,0.000074,0.861487,1.608656e-08,1.0
ARE,1.000000,1.193129e-11,0.001274,3.097260e-03,0.000000,7.305662e-13,6.226970e-10,7.910956e-14,4.081885e-07,6.883340e-09,...,0.001442,0.152288,6.141508e-13,0.6,0.005523,0.000956,0.010053,0.847712,3.343005e-08,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,0.991424,1.193129e-11,0.000001,0.000000e+00,0.000000,3.320826e-12,9.376938e-11,1.550985e-12,1.210569e-14,5.059047e-09,...,0.000180,0.944599,7.902043e-13,0.6,0.000164,0.000129,0.000036,0.055401,9.483905e-09,2.0
YEM,0.707885,1.099000e-11,0.000054,3.162703e-05,0.000000,5.990392e-12,3.158448e-08,2.754064e-13,3.485858e-15,1.672468e-07,...,0.020430,0.723078,8.698760e-13,0.4,0.029715,0.002088,0.012888,0.276922,5.709917e-08,3.0
ZAF,0.839193,1.026419e-11,0.002557,4.801899e-04,0.176255,1.090694e-11,1.334899e-07,1.323478e-12,2.662794e-14,1.673494e-06,...,0.021676,0.382063,6.661051e-13,0.2,0.068923,0.008304,0.046438,0.617937,3.679727e-08,1.0


In [81]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import set_link_color_palette
kmeans_Hierarchical(wdi_scaled)

ValueError: The number of observations cannot be determined on an empty distance matrix.

In [78]:
model = LogisticRegression()
rfe = RFE(estimator = model, n_features_to_select = 7)
X_rfe = rfe.fit_transform(X = wdi_scaled.iloc[:,:-1], y = wdi_scaled[['Dualcit_grouped']])

selected_features = pd.Series(rfe.support_, index = wdi_scaled.iloc[:,:-1].columns)
rank = pd.Series(rfe.ranking_, index = wdi_scaled.iloc[:,:-1].columns)

features_rfe = pd.concat([selected_features,rank],axis=1)
features_rfe.sort_values(1).head(20)

,0,1
Access to electricity (% of population),True,1
"Secondary education, duration (years)",True,1
Rural population,True,1
"Population, total",True,1
Population growth (annual %),True,1
Agricultural land (% of land area),True,1
Forest area (% of land area),True,1
Capture fisheries production (metric tons),False,2
Urban population growth (annual %),False,3
Agricultural land (sq. km),False,4


In [115]:
f_st = pd.Series(f_classif(wdi_scaled.iloc[:,:-1], wdi_scaled[['Dualcit_grouped']])[0],index=wdi_scaled.iloc[:,:-1].columns,name='f_statistic')
p_val = pd.Series(f_classif(wdi_scaled.iloc[:,:-1], wdi_scaled[['Dualcit_grouped']])[1],index=wdi_scaled.iloc[:,:-1].columns,name='p_value')

pd.concat([f_st,p_val],axis=1).sort_values('p_value')

,f_statistic,p_value
"Exports of goods, services and primary income (BoP, current US$)",5.107599,0.006908
Urban population growth (annual %),5.067053,0.007179
Population growth (annual %),5.067053,0.007179
"Exports of goods and services (BoP, current US$)",5.034199,0.007407
"Imports of goods, services and primary income (BoP, current US$)",4.921402,0.008245
...,...,...
External health expenditure (% of current health expenditure),0.177926,0.837143
Households and NPISHs final consumption expenditure (% of GDP),0.174362,0.840127
"Services, value added (current LCU)",0.170579,0.843305
Surface area (sq. km),0.146447,0.863869


In [98]:
X_train, X_val, y_train, y_val = train_test_split(wdi_scaled.iloc[:,:-1],wdi_scaled[['Dualcit_grouped']], test_size = 0.3, random_state = 0, stratify = wdi_scaled[['Dualcit_grouped']], shuffle = True)

#no of features
nof_list=np.arange(1,50)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    model = LogisticRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_val_rfe = rfe.transform(X_val)
    model.fit(X_train,y_train)
    
    score = model.score(X_val,y_val)
    score_list.append(score)
    
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
        
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

Optimum number of features: 1
Score with 1 features: 0.655172


# Merging the tables

In [238]:
def merge_tables(df,indicator_name):
    
    wdi_indicator = wdi.loc[wdi['Indicator Name']==indicator_name].set_index('Country Code').T.iloc[3:-1,:].reset_index()
    wdi_indicator['index'] = wdi_indicator['index'].astype(int)
    wdi_indicator = wdi_indicator.loc[wdi_indicator['index']>=2000]
    
    year = np.array([])
    ISOCODE = np.array([])
    value =  np.array([])
    
    for ISO in wdi_indicator.columns[1:-1]:
        ISOCODE_ = np.full((len(wdi_indicator),),ISO)
        ISOCODE = np.append(ISOCODE,ISOCODE_)
        
        year = np.append(year,wdi_indicator['index'].values)
        value = np.append(value,wdi_indicator[ISO].values)

        
    ISOCODE_SER = pd.Series(ISOCODE)
    year_ser = pd.Series(year)
    value = pd.Series(value).astype(float)
    final_wdi = pd.DataFrame([ISOCODE_SER,year_ser,value]).T.rename(columns={0:'ISO3',1:'Year',2:indicator_name})
    final_wdi['Year'] = final_wdi['Year'].astype(int)
    df_merge = df.merge(final_wdi, right_on=['ISO3','Year'],left_on = ['ISO3','Year'] )
    return   df_merge
    

In [239]:
#create a list of indicators names to merge with the dual cit base

df_merge = df.copy()
for ind in Indicator_little_nan[0:2]:
    df_merge = merge_tables(df_merge,ind)